# Pandas0.25来了千万别错过这10大好用的新功能

# 1 四个置顶的警告！
> 1. 从 0.25 起，pandas 只支持 Python 3.53 及以上版本了，不再支持 Python 2.7
> 2. 下一版 pandas 将只支持 Python 3.6 及以上版本了，这是因为 f-strings 的缘故
> 3. 去掉了 Panel，N 维数据结构以后要用 xarray 了
> 4. read_pickle() 与 read_msgpack() ，只向后兼容到 0.20.3

# 新增功能
# 2.1 Groupby 的 命名聚合（Named Aggregation）

In [2]:
from pyecharts import options as opts
from pyecharts.charts import Bar,Line,Pie
opts.ToolboxOpts(True)
from pyecharts.globals import CurrentConfig
CurrentConfig.ONLINE_HOST = "http://172.25.21.22:8888/static/components/pyecharts/assets/"

import pandas as pd
import numpy as np
import eplot.eplot

In [3]:
pd.__version__

'0.25.0'

In [4]:
animals = pd.DataFrame({ 
    '品种' : [ '猫' , '狗' , '猫' , '狗', '狗'],   
    '身高' : [ 9.1 , 6.0 , 9.5 , 34.0 , 35.0],   
    '体重' : [ 7.9 , 7.5 , 9.9 , 198.0, 8.2]})
animals

,品种,身高,体重
0,猫,9.1,7.9
1,狗,6.0,7.5
2,猫,9.5,9.9
3,狗,34.0,198.0
4,狗,35.0,8.2


In [5]:
animals.groupby( '品种').agg(
    最低=pd.NamedAgg(column='身高' , aggfunc= 'min'),
    最高=pd.NamedAgg(column='身高' , aggfunc= 'max'),
    平均体重=pd.NamedAgg(column='体重', aggfunc=np.mean),
)

,最低,最高,平均体重
品种,,,
狗,6.0,35.0,71.233333
猫,9.1,9.5,8.900000


In [8]:
animals.groupby( '品种').agg(最低=('身高', min),最高=('身高', max),平均体重=('体重', np.mean),)

,最低,最高,平均体重
品种,,,
狗,6.0,35.0,71.233333
猫,9.1,9.5,8.900000


In [25]:
animals.groupby( '品种').身高.agg(最低=min,最高=max,平均=np.mean)

,最低,最高,平均
品种,,,
狗,6.0,35.0,25.0
猫,9.1,9.5,9.3


In [21]:
animals.groupby( '品种').身高.count()

品种
狗    3
猫    2
Name: 身高, dtype: int64

In [36]:
animals.groupby( '品种').agg({'身高': "count",'体重': lambda x: x.iloc[ 0 ]})

,身高,体重
品种,,
狗,3,7.5
猫,2,7.9


## 2.2 Groupby 聚合支持多个 lambda 函数

In [46]:
animals.groupby( '品种' ).身高.agg(第一条的值= lambda x: x.iloc[ 0 ], 最后一条的值=lambda x: x.iloc[ -1])

,第一条的值,最后一条的值
品种,,
狗,6.0,35.0
猫,9.1,9.5


In [43]:
a=animals.groupby( '品种' ).agg(身高差=('身高',lambda x: x.iloc[ 0 ] - x.iloc[ 1 ]),体重差=('体重',lambda x: x.iloc[ 0 ] - x.iloc[ 1]))
a

,身高差,体重差
品种,,
狗,-28.0,-190.5
猫,-0.4,-2.0


In [101]:
b=animals.groupby( '品种' ).agg([lambda x: x.iloc[ 0 ] - x.iloc[ 1 ],lambda x: x.iloc[ 0 ] + x.iloc[ 1]])
idx=pd.MultiIndex(levels=[['身高', '体重'], ['差', '和']],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['类别1', '类别2'])
b.columns=idx
b

类别1    身高           体重       
类别2     差     和      差      和
品种                           
狗   -28.0  40.0 -190.5  205.5
猫    -0.4  18.6   -2.0   17.8

In [104]:
b['身高'][['差','和']]

类别2,差,和
品种,,
狗,-28.0,40.0
猫,-0.4,18.6


In [113]:
b.loc['狗']['身高']['差']

-28.0

## 2.3 优化了 MultiIndex 显示输出

In [68]:
c=pd.MultiIndex.from_product([[ 'a' , 'abc' ], range( 500 )])
c

MultiIndex([(  'a',   0),
            (  'a',   1),
            (  'a',   2),
            (  'a',   3),
            (  'a',   4),
            (  'a',   5),
            (  'a',   6),
            (  'a',   7),
            (  'a',   8),
            (  'a',   9),
            ...
            ('abc', 490),
            ('abc', 491),
            ('abc', 492),
            ('abc', 493),
            ('abc', 494),
            ('abc', 495),
            ('abc', 496),
            ('abc', 497),
            ('abc', 498),
            ('abc', 499)],
           length=1000)

## 2.4 精简显示 Series 与 DataFrame
> * 超过 60 行的 Series 与 DataFrame，pandas 会默认最多只显示 60 行（见 display.max_rows 选项）。这种设置依然会占用大量垂直屏幕空间。因此，0.25 版引入了 display.min_rows 选项，默认只显示 10 行
> * 最大与最小行数这种双重选项，允许在数据量较小时，比如数据量少于 60 行，显示全部数据，在数据量较大时，则只显示数据摘要。

In [74]:
pd.options.display.min_rows = 10
pd.options.display.max_rows = 10
sales_date1 = pd.date_range( '20190101' , periods=1000, freq= 'D'
)
amount1 = np.arange(1000)
cols = ['销售金额']
sales1 = pd.DataFrame(amount1,index=sales_date1,columns=cols)
sales1

,销售金额
2019-01-01,0
2019-01-02,1
2019-01-03,2
2019-01-04,3
2019-01-05,4
...,...
2021-09-22,995
2021-09-23,996
2021-09-24,997
2021-09-25,998


## 2.5  json_normalize() 支持 max_level
> * 0.25 以后是这样的，可以通过 max_level 参数控制读取的 JSON 数据层级

In [116]:
from pandas.io.json import json_normalize
data = [{
'CreatedBy' : { 'Name' : 'User001' },
'Lookup' : { 'TextField' : 'Some text' , 'UserField' : { 'Id' : 'ID001' , 'Name' : 'Name001' }},
'Image' : { 'a' : 'b'}
}]
json_normalize(data)

,CreatedBy.Name,Lookup.TextField,Lookup.UserField.Id,Lookup.UserField.Name,Image.a
0,User001,Some text,ID001,Name001,b


In [117]:
json_normalize(data, max_level= 1 )

,CreatedBy.Name,Lookup.TextField,Lookup.UserField,Image.a
0,User001,Some text,"{'Id': 'ID001', 'Name': 'Name001'}",b


## 2.6 增加 explode() 方法，把 list “炸 ” 成行

In [122]:
df = pd.DataFrame([{ '变量1' : 'a,b,c' , '变量2' : 1},{'变量1' : 'd,e,f' , '变量2' : 2}])
df.assign(变量1 =df.变量1. str.split( ',' )).explode( '变量1' )

,变量1,变量2
0,a,1
0,b,1
0,c,1
1,d,2
1,e,2
1,f,2


## 2.7 SparseDataFrame 被废弃了
> 0.25 以前专门有 SparseDataFrame() ，生成稀疏矩阵，0.25 以后，这个函数被废弃了，改成 pd.DataFrame 里的 pd.SparseArray() 了，函数统一了，但是要多敲几下键盘了。

In [123]:
pd.SparseDataFrame({ "A" : [0, 1]})

/data1/jiadu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  """Entry point for launching an IPython kernel.
/data1/jiadu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3451: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return klass(values, index=self.index, name=items, fastpath=True)


,A
0,0
1,1


In [124]:
pd.DataFrame({ "A" : pd.SparseArray([0, 1])})

,A
0,0
1,1


## 2.8 对 DataFrame Groupby 后，Groupby.apply 对每组只处理一次
> 0.25 以前输出的结果,针对x会输出多次

In [133]:
df = pd.DataFrame({ 'a' : [ 'x','y','x','y','x'],'b' : [ 1 , 2,2,3,5]})
df
def func(group): 
    print(group) 
    print('===')
    return group
df.groupby('a').apply(func)

   a  b
0  x  1
2  x  2
4  x  5
===
   a  b
1  y  2
3  y  3
===


,a,b
0,x,1
1,y,2
2,x,2
3,y,3
4,x,5


## 2.9 用 Dict 生成的 DataFrame，终于支持列排序啦
> 0.25 以前,列的顺序不是按字典属性的顺序

In [157]:
data = [{'姓 名' : '张三' , '城 市' : '北京' , '年 龄' : 18},
        {'姓 名' : '李四' , '城 市' : '上海' , '年 龄' : 19 , '爱 好' : '打游戏'},
        {'姓 名' : '王五' , '城 市' : '广州' , '年 龄' : 20 , '财务状况' : '优'}
       ]
pd.DataFrame(data)

,姓 名,城 市,年 龄,爱 好,财务状况
0,张三,北京,18,NaN,NaN
1,李四,上海,19,打游戏,NaN
2,王五,广州,20,NaN,优


## 2.10 Query() 支持列名空格了
> 用上面的 data 生成一个示例 DataFrame，注意列名是有空格的。
> df = pd.DataFrame(data)
> 现在用反引号（\`）括住列名，就可以直接查询了：
> df.query( '\`年 龄\` <19' )

In [158]:
df=pd.DataFrame(data)
df.query("`年 龄` <19")

,姓 名,城 市,年 龄,爱 好,财务状况
0,张三,北京,18,NaN,NaN


In [165]:
data2 = [{'姓名' : '张三' , '城市' : '北京' , '年龄' : 18},
        {'姓名' : '李四' , '城市' : '上海' , '年龄' : 19 , '爱 好' : '打游戏'},
        {'姓名' : '王五' , '城市' : '广州' , '年龄' : 20 , '财务状况' : '优'}
       ]
df2=pd.DataFrame(data2)
df2.query("`城市` == '北京' ")

,姓名,城市,年龄,爱 好,财务状况
0,张三,北京,18,NaN,NaN
